# Best Practices for Prompt Engineering



In [2]:
!pip install openai tiktoken python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.8 MB/s eta 0:00:00


In [ ]:
import os

# Definir las variables de entorno dentro de Google Collab
os.environ["AZURE_OPENAI_ENDPOINT"] = "YOUR_ENDPOINT"
os.environ["AZURE_OPENAI_API_KEY"] = "YOUR_KEY"

In [4]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import openai
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = "gpt-4o-mini"

# 1. Use the latest model

Use the latest model for best results.

# 2. Put instructions at the begining of the prompt and use ### or """ to separate the instruction and context

In [5]:
# Define las instrucciones que guiarán al modelo para que resuma el texto dado, como una lista de viñetas.
# Luego, utiliza esas instrucciones para construir un "prompt" (entrada) que incluye el contexto del texto a resumir.

instrucctiones = 'Resume el texto dado, como una lista de viñetas. \n\n '

prompt = f"{instrucctiones} ###\n\nWe’re happy to announce that OpenAI and Microsoft are extending our partnership.\
        This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
        in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
        increasingly safe, useful, and powerful. \n\n \
        In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
        capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
        raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
        broadly sharing benefits and the need to prioritize safety. \
        Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n###"
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt}],
        max_tokens=400,)

print(response.choices[0].message.content)

- OpenAI and Microsoft are extending their partnership.
- This is a multi-year, multi-billion dollar investment from Microsoft.
- Microsoft previously invested in OpenAI in 2019 and 2021.
- The investment will support OpenAI's independent research.
- The goal is to develop AI that is safe, useful, and powerful.
- OpenAI is a capped-profit company governed by a non-profit structure.
- This structure enables OpenAI to raise capital while prioritizing safety and sharing benefits.
- Microsoft shares OpenAI's vision and values, making their partnership vital for progress.


# 3. Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc

In [ ]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a poem about OpenAI.',}],
        max_tokens=400,)

print(response.choices[0].message.content)


In the heart of code where knowledge streams,  
A spark ignites, transforming dreams.  
Words that dance on silken threads,  
OpenAI, where curiosity spreads.  

From circuits deep and data wide,  
A whisper of wisdom, a guide inside.  
Crafting thoughts from ones and zeroes,  
A tapestry woven of digital heroes.  

With language as art and logic as light,  
We explore the shadows and reach for the bright.  
In queries and answers, we find our way,  
A bridge to the future, come what may.  

In the realm of the mind, where limits persist,  
You broaden horizons, uncoil the mist.  
From science to stories, from math to the muse,  
A wellspring of wonders, a world we can use.  

Yet with great power comes duty to heed,  
For ethics must flourish in every deed.  
To guide our journey, to learn and to grow,  
To harness the knowledge for all, and to show.  

So here's to the vision that sparks innovation,  
To those who create and inspire transformation.  
OpenAI, a beacon, forever to stri

In [ ]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a short inspiring poem about OpenAI, \
                focusing on the recent DALL-E product launch in the style of Ernest Hemingway',}],
        max_tokens=400,)

print(response.choices[0].message.content)



In quiet dawn, the canvas waits,  
Blank as a thought, untouched by light,  
With strokes of dreams, the artist creates,  
A world where visions take their flight.  

DALL-E stirs, born of code and spark,  
Crafting wonders from whispers of mind,  
Colors collide in a dance, a lark,  
Images forged, new realms defined.  

Simplicity sings, yet depth runs deep,  
Each pixel a promise, each frame a chance,  
OpenAI, a light in shadows steep,  
Invites the heart to rise and dance.  

In the art of the unseen, we find our way,  
Where words fade, and visions stay.  


# 4. Articulate the desired output format through examples (example 1, example 2).

In [ ]:
#Escribe las instrucciones aquí. Las instrucciones deben especificar que el modelo debe:
# 1. Extraer los nombres de las empresas y los años del texto.
# 2. Generar la salida como un diccionario con 'text', 'start' y 'end'.
instrucctiones = ' ...(Extract the companies name and the years from the text)... \
                    Generate output as {{\"text\": \"OpenAI\", \"start\": 28, \"end\": 34}}' #FIXME

prompt = f"{instrucctiones}  \
        ### We’re happy to announce that OpenAI and Microsoft are extending our partnership. This multi-year, \
        multi-billion dollar investment from Microsoft follows their previous investments in 2019 and 2021, \
        and will allow us to continue our independent research and develop AI that is increasingly safe, useful, and powerful. \n\n ### "


response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



```json
[
    {"text": "OpenAI", "start": 28, "end": 34},
    {"text": "Microsoft", "start": 43, "end": 52},
    {"text": "2019", "start": 90, "end": 94},
    {"text": "2021", "start": 99, "end": 103}
]
```


In [ ]:

# Las instrucciones deben pedir al modelo que extraiga los siguientes tipos de entidades del texto:
# 1. Nombres de empresas.
# 2. Años.
# 3. Temas específicos relacionados con el contenido.
# 4. Temas generales y abarcativos.
# El formato de salida esperado debe incluir listas separadas por comas para cada tipo de entidad.

instrucctiones = 'Extract the following entities from the text: Companies name, years, topics related with the content, general and comprehensive topics. \n\n'
instrucctiones+='The output should include a list separated by commas \n\n'

prompt = f"{instrucctiones}  \
        '\
                We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                '"

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



Companies names: OpenAI, Microsoft  
Years: 2019, 2021  
Topics related with the content: partnership, investment, artificial intelligence, research, safety, usefulness, power  
General and comprehensive topics: technology, business, innovation


# 5. Start with zero-shot, then few-shot (example), neither of them worked, then fine-tune (To update)

In [ ]:
#THIS IS zero-shot

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below."},
                {"role":"user","content": 'Text: \n\
            We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
            This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
            in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
            increasingly safe, useful, and powerful. \n\nKeywords:    ',}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, partnership, multi-year, multi-billion dollar investment, research, AI, safe, useful, powerful.


In [ ]:
#Few Shot learning


# Escribir las instrucciones para que el modelo extraiga palabras clave del texto, y formatee la salida como listas separadas por comas.

instrucctiones = 'Extract key words from the text and format as lists separated by commas. \n\n'

prompt = f"{instrucctiones} \
                Text: Stripe provides APIs that web developers can use to integrate \
                payment processing into their websites and mobile applications. \
                Keywords: Stripe, payment processing, APIs, web developers, websites, mobile applications \
                ###\n\
                Text: OpenAI has trained cutting-edge language models that are very good at understanding \
                and generating text. Our API provides access to these models and can be used to solve virtually \
                any task that involves processing language. \n\
                Keywords: language models, text processing, API.\n\n\
                ##W"

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":prompt},
                {"role":"user","content": '\n\
                Text: We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n\
                Keywords:',}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, partnership, investment, independent research, AI, safe, useful, powerful.


# 6. Reduce “fluffy” and imprecise descriptions

In [ ]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                The description for this product should be fairly short, a few sentences only, and not too much more.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Introducing the NextGen SafeRide Car Seat — a revolutionary blend of safety, comfort, and style. Engineered with adaptive cushioning and intelligent safety sensors, this innovative car seat ensures your child is secure and supported during every ride. With an easy installation system and a sleek design that complements any vehicle interior, the SafeRide Car Seat is the ultimate choice for modern parents seeking peace of mind on the road. Elevate your travel experience with cutting-edge technology and unparalleled protection.


In [6]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                Use a 3 to 5 sentence paragraph to describe this product.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Introducing the EcoComfort SmartCar Seat, the next generation of car safety and comfort designed for families on the go. With its innovative adaptive design, this car seat features adjustable support that grows with your child, ensuring optimal safety from infancy through toddlerhood. The built-in smart technology monitors your child’s posture and temperature, providing real-time adjustments and alerts to keep them comfortable and secure. Crafted from eco-friendly materials, it’s not just a car seat; it’s a commitment to sustainability and peace of mind for every journey. Rediscover road trips with the EcoComfort SmartCar Seat, where safety meets sophistication.


# 7. Instead of just saying what not to do, say what to do instead

In [7]:
response = client.chat.completions.create(
    model = CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'The following is a conversation between an Agent and a Customer. DO NOT ASK USERNAME OR PASSWORD. DO NOT REPEAT. \n\n\
                Customer: I can’t log in to my account.\n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging in. Can you describe any error messages you're seeing, or let me know what happens when you try to log in?


In [8]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'The following is a conversation between an Agent and a Customer. The agent will attempt to diagnose the \
                problem and suggest a solution, whilst refraining from asking any questions related to PII. \
                Instead of asking for PII, such as username or password, refer the user to the help \
                article www.samplewebsite.com/help/faq \n\n\
                Customer: I can’t log in to my account. \n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging in. There are a few common reasons why this might happen. First, I recommend checking to ensure that you are entering your login credentials correctly. Sometimes, it can be easy to mistype a password or username.

If you're still having trouble, you might want to try resetting your password. You can do this by following the instructions in our help article at www.samplewebsite.com/help/faq.

If you continue to experience issues, please refer to that article for additional troubleshooting steps or contact support for further assistance.


# 8. Code Generation Specific - Use “leading words” to nudge the model toward a particular pattern

In [ ]:
# Escribir las instrucciones para crear una función simple en Python que:
# 1. Pida un número en millas.
# 2. Convierta las millas a kilómetros.

prompt = 'Create a simple function in Python with this instructions: \n\
          1. Ask a number in miles \n\
          2.Convert miles to kilometers'

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":prompt}],
        max_tokens=400,)

print(response.choices[0].message.content)


Here is a simple Python function that follows your instructions. It prompts the user to input a number in miles and then converts it to kilometers.

```python
def miles_to_kilometers():
    # Ask the user for a number in miles
    miles = float(input("Please enter the number of miles: "))
    
    # Convert miles to kilometers (1 mile = 1.60934 kilometers)
    kilometers = miles * 1.60934
    
    # Display the result
    print(f"{miles} miles is equal to {kilometers:.2f} kilometers.")

# Call the function
miles_to_kilometers()
```

To use this function, simply run the code. It will prompt you to enter a number in miles and will then output the equivalent distance in kilometers.


In [ ]:
def miles_to_kilometers():
    # Ask the user for a number in miles
    miles = float(input("Please enter the number of miles: "))

    # Convert miles to kilometers (1 mile = 1.60934 kilometers)
    kilometers = miles * 1.60934

    # Display the result
    print(f"{miles} miles is equal to {kilometers:.2f} kilometers.")

# Call the function
miles_to_kilometers()

Please enter the number of miles: 12454
12454.0 miles is equal to 20042.72 kilometers.


## Ejercicio 1: Estructuración de Prompts con Separadores

Instrucciones: Escribe un prompt que solicite a la IA resumir el siguiente texto en una lista de puntos clave. Asegúrate de usar separadores claros (### o """).

Texto:
OpenAI y Microsoft han ampliado su colaboración. Microsoft está invirtiendo miles de millones de dólares durante varios años. Esta inversión permitirá que OpenAI continúe investigando de forma independiente y desarrolle IA segura y útil. OpenAI sigue siendo una empresa de beneficios limitados, regida por su organización sin fines de lucro.


In [27]:
texto = 'OpenAI y Microsoft han ampliado su colaboración. Microsoft está invirtiendo miles de millones de dólares durante varios años. \
 Esta inversión permitirá que OpenAI continúe investigando de forma independiente y desarrolle IA segura y útil. \
 OpenAI sigue siendo una empresa de beneficios limitados, regida por su organización sin fines de lucro.'

prompt = f"Summarize the following text in a key points list: {texto} "

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt}],
    max_tokens=400,
)

print(response.choices[0].message.content)



- OpenAI y Microsoft han ampliado su colaboración.
- Microsoft invertirá miles de millones de dólares durante varios años.
- La inversión permitirá a OpenAI investigar de forma independiente.
- OpenAI se enfocará en desarrollar IA segura y útil.
- OpenAI sigue siendo una empresa de beneficios limitados, bajo una organización sin fines de lucro.


## Ejercicio 2: Descripción Detallada del Formato Deseado

Instrucciones: Escribe un prompt para que la IA extraiga nombres de empresas y años del siguiente texto. Indica explícitamente que se desea el resultado en formato JSON con índices de inicio y fin.

Texto:
OpenAI y Microsoft han colaborado desde 2019, con inversiones adicionales en 2021 para desarrollar nuevas soluciones de IA.

Formato de salida deseado:

{"text": "OpenAI", "start": 0, "end": 6}
{"text": "Microsoft", "start": 10, "end": 19}
{"text": "2019", "start": 35, "end": 39}
{"text": "2021", "start": 50, "end": 54}

In [26]:
texto1 = 'OpenAI y Microsoft han colaborado desde 2019, con inversiones adicionales en 2021 para desarrollar nuevas soluciones de IA.'

prompt = f"Extract the companies name and years from the following text: {texto1} \n\
The result should be in JSON format with start and end indexes: "

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are an entity extraction assistant."},
                {"role":"user","content": prompt }],
    max_tokens=400,
)

print(response.choices[0].message.content)


```json
{
  "companies": [
    {
      "name": "OpenAI",
      "start_index": 0,
      "end_index": 6
    },
    {
      "name": "Microsoft",
      "start_index": 10,
      "end_index": 18
    }
  ],
  "years": [
    {
      "year": "2019",
      "start_index": 30,
      "end_index": 34
    },
    {
      "year": "2021",
      "start_index": 50,
      "end_index": 54
    }
  ]
}
```


## Ejercicio 3: Generación de Texto Inspirador

Instrucciones: Redacta un prompt que solicite a la IA escribir un poema inspirador. Escribe un poema corto sobre la colaboración entre OpenAI y Microsoft.


In [20]:
prompt = "Write an inspirational short poem about the collaboration between OpenAI and Microsoft"

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a creative assistant."},
                {"role":"user","content": prompt}],
    max_tokens=400,
)

print(response.choices[0].message.content)

In a world of code and dreams that soar,  
Two minds entwined, they seek to explore.  
OpenAI’s spark, with wisdom aglow,  
Microsoft's reach, together they'll grow.  

With silicon hearts and visions so bright,  
They craft a tomorrow, igniting the night.  
Through lines of soft code, innovation will thrive,  
In partnership strong, they breathe life, they drive.  

From cloud-kissed heights to the depths of the sea,  
A tapestry woven of possibility.  
Together they rise, pushing boundaries wide,  
In the dance of creation, they stand side by side.  

So let each idea take flight on its wings,  
With unity strong, see what wonder it brings.  
For in collaboration, the future’s embraced,  
A symphony of progress, beautifully paced.  


## Ejercicio 4: Extracción de Entidades Clave

Instrucciones: Escribe un prompt que pida a la IA extraer nombres de empresas, años, temas específicos y temas generales del siguiente texto:

Texto:
OpenAI y Microsoft han trabajado juntos desde 2019 y reforzaron su colaboración en 2021


Formato esperado:

Company names: OpenAI, Microsoft \
Years: 2019, 2021 \
Specific topics: colaboración \
General themes: asociación, tecnología, inteligencia artificial

In [19]:
text = 'OpenAI y Microsoft han trabajado juntos desde 2019 y reforzaron su colaboración en 2021'

prompt = f"Extract company names, years data, specific topics and general topics from the following text: \n\\n\ {text} \n\\n\
Expected format: \n\
Company names: \
Years: \
Specific topics: \
General themes: "

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are an entity and theme extraction assistant."},
                {"role":"user","content": prompt}],
    max_tokens=400,
)

print(response.choices[0].message.content)


Company names: OpenAI, Microsoft  
Years: 2019, 2021  
Specific topics: Collaboration  
General themes: Partnership, Technology


## Ejercicio 5: Descripciones Concisas de Productos

Instrucciones: Escribe un prompt para que la IA describa un nuevo tipo de mochila tecnológica en menos de 4 líneas. La mochila tiene cargador solar, materiales resistentes al agua y un diseño ergonómico.



In [10]:
prompt = "Describe a new technology backpack prototype with a maximum of 4 lines."

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a marketing assistant."},
                {"role":"user","content": prompt}],
    max_tokens=400,
)

print(response.choices[0].message.content)


Introducing the TechFlux Backpack: a sleek, anti-theft design equipped with integrated wireless charging pads, solar panels for on-the-go power, and Bluetooth tracking for lost items. It features customizable compartments with modular dividers, and a built-in smart display that syncs with your devices to show notifications and battery levels. Perfect for tech-savvy travelers and students alike!
